# Ploting sample of data

In [ ]:
def get_labels(image_id):
    ''' Function to get the labels for the image by name'''
    im_df = train_df[train_df['Image'] == image_id].fillna('-1')
    im_df = im_df[im_df['EncodedPixels'] != '-1'].groupby('Label').count()

    index = im_df.index
    all_labels = ['Fish', 'Flower', 'Gravel', 'Sugar']

    labels = ''

    for label in all_labels:
        if label in index:
            labels = labels + ' ' + label

    return labels

### 2.plotting 5 samples

In [ ]:
# lets group each of the types and their mask in a list so we can do more aggregated counts
grouped_EncodedPixels = train_df.groupby('ImageId')['Label_EncodedPixels'].apply(list)

In [ ]:
import numpy as np

def bounding_box(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)

    rmin, rmax = (0, 0)  # Default values for empty masks
    cmin, cmax = (0, 0)  # Default values for empty masks

    if np.any(rows):
        rmin, rmax = np.where(rows)[0][[0, -1]]

    if np.any(cols):
        cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax


In [ ]:
def plot_cloud(img_path, img_id, label_mask):
    img = cv2.imread(os.path.join(img_path, img_id))

    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(20,10))
    ax[0].imshow(img)
    ax[1].imshow(img)
    cmaps = {'Fish': 'Blues', 'Flower': 'Reds', 'Gravel': 'Greys', 'Sugar':'Purples'}
    colors = {'Fish': 'Blue', 'Flower': 'Red', 'Gravel': 'Gray', 'Sugar':'Purple'}
    for label, mask in label_mask:
        mask_decoded = rle_to_mask(mask, img.shape[0], img.shape[1])
        if mask != -1:
            rmin, rmax, cmin, cmax = bounding_box(mask_decoded)
            bbox = patches.Rectangle((cmin,rmin),cmax-cmin,rmax-rmin,linewidth=1,edgecolor=colors[label],facecolor='none')
            ax[0].add_patch(bbox)
            ax[0].text(cmin, rmin, label, bbox=dict(fill=True, color=colors[label]))
            ax[1].imshow(mask_decoded, alpha=0.3, cmap=cmaps[label])
            ax[0].text(cmin, rmin, label, bbox=dict(fill=True, color=colors[label]))

In [ ]:
for image_id, label_mask in grouped_EncodedPixels.sample(5).iteritems():
    plot_cloud(TRAIN_PATH, image_id, label_mask)